# IMPORT PACKAGES

In [1]:
import os
import requests
import urllib
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.animation as animation
from matplotlib.offsetbox import OffsetImage,AnnotationBbox
matplotlib.rcParams['animation.embed_limit'] = 10000
import numpy as np

from tqdm import tqdm_notebook as tqdm
from IPython.display import display, HTML
from datetime import datetime
ymd = datetime.now().strftime('%y%m%d')

# DOWNLOAD DATA

In [4]:
#hide
url_confirms = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

def get_frame(url):
    df = pd.read_csv(url)
    return df

def extractGlobal(df):
    df = df.drop(columns=['Province/State', 'Lat', 'Long'])
    df = df.groupby('Country/Region').sum()
    return df

confirmedRaw = get_frame(url_confirms)
deaths = get_frame(url_deaths)
confirm_df = extractGlobal(confirmedRaw)
deaths_df = extractGlobal(deaths)
display(confirm_df.head())
display(deaths_df.head())

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,22,22,22,24,24,40,40,74,84,94
Albania,0,0,0,0,0,0,0,0,0,0,...,55,59,64,70,76,89,104,123,146,174
Algeria,0,0,0,0,0,0,0,0,0,0,...,60,74,87,90,139,201,230,264,302,367
Andorra,0,0,0,0,0,0,0,0,0,0,...,39,39,53,75,88,113,133,164,188,224
Angola,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,2,2,3,3,3,4


,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,2,4
Albania,0,0,0,0,0,0,0,0,0,0,...,1,2,2,2,2,2,4,5,5,6
Algeria,0,0,0,0,0,0,0,0,0,0,...,4,7,9,11,15,17,17,19,21,25
Andorra,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,3
Angola,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def get_interpolate_df(input_df, nb_copies = 50):
    '''
    Linear Interpolate between every two columns for better visualization and smooth transition
    '''
    df = input_df
    i_df = pd.DataFrame()
    columns = df.columns.tolist()
    last_col = -1
    samples = nb_copies
    for idx, col in enumerate(columns[:last_col]):
        i_df[col] = df[col]
        new_columns = ['{}'.format(i) for i in range(idx*samples,idx*samples+samples)]
        for row in range(len(df.index)):
            i_vec = np.linspace(df.iloc[row,idx], df.iloc[row,idx+1], samples, dtype = np.int32)
            i_mat = i_vec if row == 0 else np.vstack((i_mat, i_vec))
        for j, jcol in enumerate(new_columns[1:-2]):
            i_df[jcol] = i_mat[:,j]
    i_df[columns[last_col]] = df[columns[last_col]]
    return i_df

# Download Country flags

In [ ]:
index = [index.replace(',','').replace('*','') for index in df.index]
df.index = index
index = [index.replace(',','').replace('*','') for index in death_df.index]
death_df.index = index
flag_df = pd.read_csv('country_flag.csv')
for idx, country in tqdm(enumerate(df.index)):
    filepath = 'countryflags/{}.png'.format(country)
    url = df['Image URL'].iloc[idx]
    if not os.path.isfile(filepath):
        try:
            r = requests.get(url, stream = True)
            if r.status_code == 200:
                with open(filepath, 'wb') as f:
                    shutil.copyfileobj(r.raw, f)
                del r
        except:
            print('Failed: {}'.format(url))

In [ ]:
# Interpolate DataFrame
i_df = get_interpolate_df(df)
death_idf = get_interpolate_df(death_df)
display(i_df.head())

# PLOT

In [ ]:
cmap = matplotlib.cm.get_cmap('tab20', len(df.index))    # PiYG
color_list = []
for i in range(cmap.N):
    rgb = cmap(i)[:3] # will return rgba, we take only first 3 so we get rgb
    color_list.append(matplotlib.colors.rgb2hex(rgb))
colors = colors = dict(zip(df.index.unique(), color_list))

def draw_barchart(col_idx):
    dff = i_df.iloc[:,col_idx].sort_values(ascending = True).tail(10)
    death_dff = death_idf.iloc[:,col_idx].loc[dff.index]
    ax.clear()
    ax.barh(dff.index, dff.values, height = 0.5, color = [colors[x] for x in dff.index], alpha = 0.5)
    ax.barh(death_dff.index, death_dff.values, height = 0.4, 
            color = [colors[x] for x in dff.index], alpha = 1.)
    for idx, (country, cases) in enumerate(zip(dff.index, dff.values)):
        path = "countryflags/{}.png".format(country)
        try:
            im = plt.imread(path)
        except:
            im = np.zeros((16,16))
        im = OffsetImage(im, zoom=0.5)
        ab = AnnotationBbox(im, (cases,idx), frameon = False, box_alignment = (1.2, 0.5))
        ax.add_artist(ab)
        ax.text(cases, idx, '   {:,.0f}'.format(cases),  size=14, ha='left',  va='center')
        if idx == len(dff.index)-1:
            ax.text(cases, idx+0.2, 'Cases',  size=14, ha='center',  va='bottom')
    
    for idx, (country, deaths) in enumerate(zip(death_dff.index, death_dff.values)):
        if idx == len(death_dff.index)-1:
            ax.text(deaths, idx+0.2, r'${:,.0f}^* Deaths$'.format(deaths),  size=14, ha='left',  va='bottom')
        else:
            ax.text(deaths, idx+0.2, r'${:,.0f}^*$'.format(deaths),  size=14, ha='left',  va='bottom')
    date_text = str(df.columns[int(np.floor(col_idx/(samples-2)))])
    ax.text(1, 0.15, date_text, transform=ax.transAxes, color='#777777', size=46, ha='right', weight=800)
    ax.text(1, 0.08, 'Total Cases: {:,}'.format(i_df.iloc[:,col_idx].sum()), 
            transform=ax.transAxes, color='#777777', size=24, ha='right', weight=800)
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.15, 'COVID19 Global Cases',
            transform=ax.transAxes, size=32, weight=600, ha='left', va='top')
    ax.tick_params(axis='y', which='major', pad=40)
    plt.annotate('Latest data from {}\n'
             'Data Source: Johns Hopkins CSSE\n'
             'Chart by Phat Luu'.format(i_df.columns[-1]), 
             (0.95, 0.03), xycoords='figure fraction', fontsize=12, ha = 'right');
    fontproperties = {'weight': 'bold'}
    plt.box(False)
    
fig, ax = plt.subplots(figsize=(16, 9))

# EXPORT VIDEO

In [ ]:
animator = animation.FuncAnimation(fig, draw_barchart, frames = range(len(i_df.columns)), interval = 10)
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='PhatLuu'))
animator.save('covid19_global.mp4', writer=writer)
print('DONE!')